In [112]:
from asf_hyp3 import API
from collections import namedtuple
# import pandas as pd
from src.api_functions import hyp3_login, grab_subscription
from pprint import pprint
from datetime import date, datetime
import re
from pathlib import Path
import os
import dataclasses as dc
import numpy as np
from dataclasses import field, asdict
from shapely import wkt
from shapely.geometry import Polygon
import requests
import zipfile
import shutil
from tempfile import TemporaryDirectory
from itertools import groupby
import os
import itertools
from osgeo import gdal
from src.model import load_model
from src.config import NETWORK_DEMS as dems
import pathlib

## Product Class

In [62]:
@dc.dataclass()
class Product:
    name: str
    granule: str = None
    url: str = None
    shape: Polygon = None
    start: datetime = None
    end: datetime = None
        
    def __post_init__(self):
        self.product_time_regex = re.compile(
                r"S.*1SDV_(?P<start_year>\d{4})(?P<start_month>\d{2})(?P<start_day>\d{2})T(?P<start_hour>\d{2})("
                r"?P<start_minute>\d{2})(?P<start_second>\d{2})_(?P<end_year>\d{4})(?P<end_month>\d{2})(?P<end_day>\d{2})T("
                r"?P<end_hour>\d{2})(?P<end_minute>\d{2})(?P<end_second>\d{2})_*")
#         self.start = make_start(name)
#         self.end = make_end(name)
        self.start = self._make_start(self.granule)
        self.end = self._make_end(self.granule)
        
        

        
    def _make_start(self,product_name) -> datetime:
        
        regex_match = re.match(self.product_time_regex, product_name)
        time_dict = regex_match.groupdict()
        for k, v in time_dict.items():
            time_dict[k] = int(v)

        return datetime(time_dict["start_year"], time_dict["start_month"], time_dict["start_day"],
                        time_dict["start_hour"], time_dict["start_minute"], time_dict["start_second"])



    def _make_end(self, product_name) -> datetime:

        regex_match = re.match(self.product_time_regex, product_name)
        time_dict = regex_match.groupdict()
        for k, v in time_dict.items():
            time_dict[k] = int(v)

        return datetime(time_dict["end_year"], time_dict["end_month"], time_dict["end_day"],
                        time_dict["end_hour"], time_dict["end_minute"], time_dict["end_second"])
    
    
    def to_json(self):
        metadata = asdict(self)
        metadata['start'] = self.start.isoformat()
        metadata['end'] = self.end.isoformat()
        metadata['shape'] = str(self.shape)
        
#         for key in list(metadata):
#             if key is datetime:
#                 metadata[key] = metadata[key].isoformat()
#                 print(f"TEST: {key}= {metadata[key]}")
        return json.dumps(metadata)
    

## Credentials namedtup

In [63]:
credentials = namedtuple('credentials', 'username password')

## Functions

In [64]:
def product_time(product_name):
    product_time_regex = re.compile(
        r"S.*1SDV_(?P<start_year>\d{4})(?P<start_month>\d{2})(?P<start_day>\d{2})T(?P<start_hour>\d{2})("
        r"?P<start_minute>\d{2})(?P<start_second>\d{2})_(?P<end_year>\d{4})(?P<end_month>\d{2})(?P<end_day>\d{2})T("
        r"?P<end_hour>\d{2})(?P<end_minute>\d{2})(?P<end_second>\d{2})_*")

    regex_match = re.match(product_time_regex, product_name)
    time_dict = regex_match.groupdict()

    # converts all dates/times values in dictionary from int to string
    for k, v in time_dict.items():
        time_dict[k] = int(v)

    start = datetime(time_dict["start_year"], time_dict["start_month"], time_dict["start_day"],
                     time_dict["start_hour"], time_dict["start_minute"], time_dict["start_second"])

    end = datetime(time_dict["end_year"], time_dict["end_month"], time_dict["end_day"],
                   time_dict["end_hour"], time_dict["end_minute"], time_dict["end_second"])


    return start, end

In [65]:
def product_in_time_bounds(product_name, start, end):
    prod_start, prod_end = product_time(product_name)
    
    return prod_start > start and prod_end < end

In [66]:
def get_sub_products(api, sub_id, start, end):
    
    response = api.get_products(sub_id=sub_id)
    
    products = []
    for product in response:
        if product_in_time_bounds(product['granule'], start, end):
            products.append(Product(product['name'],
                                    product['granule'],
                                    product['url']
                                   )
                           )
    return products
    


### write_tiles

In [67]:
def write_tiles(input_tif: str, output_directory: str, tile_size: int, name: str):
    """Creates tiles of given dimension out of input tif file"""
    input_image = gdal.Open(input_tif)

    array = input_image.ReadAsArray()

    rows, cols = array.shape

    tile_indexes = itertools.product(
        range(0, rows, tile_size), range(0, cols, tile_size))

    for (row, col) in tile_indexes:
        in_bounds = row + tile_size < rows and col + tile_size < cols
        if in_bounds:
            gdal.Translate(
                f'{output_directory}{name}_x{row / tile_size}y{col / tile_size}.tif',
                input_tif,
                srcWin=[row, col, tile_size, tile_size],
                format="GTiff"
            )

### stride_tile_image

In [68]:
def stride_tile_image(

        image: np.ndarray, width: int = dems, height: int = dems
) -> np.ndarray:
    _nrows, _ncols = image.shape
    _strides = image.strides

    nrows, _m = divmod(_nrows, height)
    ncols, _n = divmod(_ncols, width)

    assert _m == 0, "Image must be evenly tileable. Please pad it first"
    assert _n == 0, "Image must be evenly tileable. Please pad it first"

    return np.lib.stride_tricks.as_strided(
        np.ravel(image),
        shape=(nrows, ncols, height, width),
        strides=(height * _strides[0], width * _strides[1], *_strides),
        writeable=False
    ).reshape(nrows * ncols, height, width)

### get_tile_dimensions

In [69]:
def get_tile_dimensions(height: int, width: int, tile_size: int):
    return int(np.ceil(height / tile_size)), int(np.ceil(width / tile_size))

### write_mask_to_file

In [70]:
def write_mask_to_file(
        mask: np.ndarray, file_name: str, projection: str, geo_transform: str
) -> None:
    (width, height) = mask.shape
    out_image = gdal.GetDriverByName('GTiff').Create(
        file_name, height, width, bands=1
    )
    out_image.SetProjection(projection)
    out_image.SetGeoTransform(geo_transform)
    out_image.GetRasterBand(1).WriteArray(mask)
    out_image.GetRasterBand(1).SetNoDataValue(0)
    out_image.FlushCache()

### pad_image

In [71]:
def pad_image(image: np.ndarray, to: int) -> np.ndarray:
    height, width = image.shape

    n_rows, n_cols = get_tile_dimensions(height, width, to)
    new_height = n_rows * to
    new_width = n_cols * to

    padded = np.zeros((new_height, new_width))
    padded[:image.shape[0], :image.shape[1]] = image
    return padded

### create_water_mask

In [72]:

def create_water_mask(
        model_path: str, vv_path: str, vh_path: str, outfile: str, verbose: int = 0
):
    if not os.path.isfile(vv_path):
        raise FileNotFoundError(f"Tiff '{vv_path}' does not exist")

    if not os.path.isfile(vh_path):
        raise FileNotFoundError(f"Tiff '{vh_path}' does not exist")

    def get_tiles(img_path):
        f = gdal.Open(img_path)
        img_array = f.ReadAsArray()
        original_shape = img_array.shape
        n_rows, n_cols = get_tile_dimensions(*original_shape, tile_size=dems)
        padded_img_array = pad_image(img_array, dems)
        invalid_pixels = np.nonzero(padded_img_array == 0.0)
        img_tiles = stride_tile_image(padded_img_array)
        return img_tiles, n_rows, n_cols, invalid_pixels, f.GetProjection(), f.GetGeoTransform()

    # Get vv tiles
    vv_tiles, vv_rows, vv_cols, vv_pixels, vv_projection, vv_transform = get_tiles(vv_path)

    # Get vh tiles
    vh_tiles, vh_rows, vh_cols, vh_pixels, vh_projection, vh_transform = get_tiles(vh_path)

    model = load_model(model_path)

    # Predict masks
    masks = model.predict(
        np.stack((vv_tiles, vh_tiles), axis=3), batch_size=1, verbose=verbose
    )

    masks.round(decimals=0, out=masks)

    # Stitch masks together
    mask = masks.reshape((vv_rows, vv_cols, dems, dems)) \
        .swapaxes(1, 2) \
        .reshape(vv_rows * dems, vv_cols * dems)  # yapf: disable

    mask[vv_pixels] = 0
    write_mask_to_file(mask, outfile, vv_projection, vv_transform)

    # Needed?
    f = None

### download_product

In [73]:
def download_product(product_url: str, save_directory: Path, creds: credentials) -> None:
    """Download sar product from given url."""

    filename = product_url.split('/')[-1]
    save_path = save_directory / filename

    # Authenticate and then get redirect
    response_redirect = requests.get(product_url)
    redirect_url = response_redirect.url

    # authenticate with .netrc and then Download fjle from redirect_url
    with requests.get(redirect_url, stream=True, auth=(creds.username, creds.password)) as response:
        with open(save_path, 'wb') as f:
            print(f"Downloading {filename}")

            # Downloads files in chucks
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
        f.close()
    

### download_products

In [74]:
def download_products(product_urls: list, output_dir: Path, creds: credentials):
    for url in product_urls:
        download_product(url, output_dir, creds)

### mask_products

In [75]:
for 

### get_netrc_credentials

In [76]:
def get_netrc_credentials() -> credentials:
    """Returns credentials from .netrc file."""

    with open('.netrc', 'r') as f:
        contents = f.read()
    username = contents.split(' ')[3]
    password = contents.split(' ')[5].split('\n')[0]


    return credentials(username, password)

### extract_from_product

In [77]:
def extract_from_product(product_path, output_dir):
    product_name = Path(product_path).stem
    sar_regex = re.compile(r"(S1[A|B])_(.{2})_(.*)_(VV|VH)(.tif)")

    with zipfile.ZipFile(product_path, "r") as zip_ref:
        with TemporaryDirectory() as tmpdir_name:
            for file_info in zip_ref.infolist():
                if re.fullmatch(sar_regex,file_info.filename):
                    zip_ref.extract(file_info,path=tmpdir_name)
                    shutil.move(f"{tmpdir_name}/{file_info.filename}", output_dir)
                    pprint(f"moved {file_info.filename} from {tmpdir_name} to {output_dir}")




# RUN

## Find the products

In [31]:
start = datetime(year=2019,month=12,day=3,hour=0,minute=0,second=0)
end   = datetime(year=2019,month=12,day=4,hour=0,minute=0,second=0)

api = hyp3_login()
id = 2810


products = get_sub_products(api, id, start, end)
product_urls = [p.url for p in products]
print(f"urls={product_urls}")


 login successful!
 Welcome jmherning
urls=['https://hyp3-download.asf.alaska.edu/asf/data/S1A_IW_20191203T224518_DVP_RTC10_G_gpuned_54B9.zip', 'https://hyp3-download.asf.alaska.edu/asf/data/S1A_IW_20191203T224543_DVP_RTC10_G_gpuned_3A53.zip']


## Download the products

In [49]:
input_directory   = Path.cwd() / "data"/ "input"
working_directory = Path.cwd() / "data"/ "working"
output_directory  = Path.cwd() / "data"/ "output"

ed_creds = get_netrc_credentials()


# Uncoment this to download again!
# download_products(product_urls, input_directory, ed_creds)
for f in input_directory.glob("*.zip"):
    print(f"this product exist! {f.name}")

this product exist! S1A_IW_20191203T224543_DVP_RTC10_G_gpuned_3A53.zip
this product exist! S1A_IW_20191203T224518_DVP_RTC10_G_gpuned_54B9.zip


## Extract vv/vh tifs

In [58]:


for f in input_directory.glob("*.zip"):
    prod_path = input_directory / f.name
    extract_from_product(prod_path, working_directory)

print(f"{os.listdir(str(working_directory)}")

('moved '
 'S1A_IW_20191203T224543_DVP_RTC10_G_gpuned_3A53/S1A_IW_20191203T224543_DVP_RTC10_G_gpuned_3A53_VH.tif '
 'from /tmp/tmp7lalua5_ to '
 '/home/jmherning/code/AI_Water/notebooks/data/working')
('moved '
 'S1A_IW_20191203T224543_DVP_RTC10_G_gpuned_3A53/S1A_IW_20191203T224543_DVP_RTC10_G_gpuned_3A53_VV.tif '
 'from /tmp/tmp7lalua5_ to '
 '/home/jmherning/code/AI_Water/notebooks/data/working')
('moved '
 'S1A_IW_20191203T224518_DVP_RTC10_G_gpuned_54B9/S1A_IW_20191203T224518_DVP_RTC10_G_gpuned_54B9_VV.tif '
 'from /tmp/tmpf681jqpj to '
 '/home/jmherning/code/AI_Water/notebooks/data/working')
('moved '
 'S1A_IW_20191203T224518_DVP_RTC10_G_gpuned_54B9/S1A_IW_20191203T224518_DVP_RTC10_G_gpuned_54B9_VH.tif '
 'from /tmp/tmpf681jqpj to '
 '/home/jmherning/code/AI_Water/notebooks/data/working')


FileNotFoundError: [Errno 2] No such file or directory: 'working'

In [137]:
TYPE_REGEX = re.compile(r"(.*)_(VV|VH).(tiff|tif|TIFF|TIF)")

vvvh_set = namedtuple('vvvh_set', ['vh', 'vv'])

def get_sar_paths(directory_path: str) -> list:
    """returns a list of namedTuples (sar_sets) that store vv, vh, and mask paths"""
    dataset_path = Path(directory_path)

    path_generator = dataset_path.rglob('*.tif')
    paths = sorted([path for path in path_generator if path.is_file()])
    return [vvvh_set(*g) for k, g in groupby(paths, key=lambda path: re.match(TYPE_REGEX, path.name)[1])]



vvvh_paths = get_sar_paths(str(working_directory))
print(sar_paths)

[vvvh_set(vh=PosixPath('/home/jmherning/code/AI_Water/notebooks/data/working/S1A_IW_20191203T224518_DVP_RTC10_G_gpuned_54B9_VH.tif'), vv=PosixPath('/home/jmherning/code/AI_Water/notebooks/data/working/S1A_IW_20191203T224518_DVP_RTC10_G_gpuned_54B9_VV.tif')), vvvh_set(vh=PosixPath('/home/jmherning/code/AI_Water/notebooks/data/working/S1A_IW_20191203T224543_DVP_RTC10_G_gpuned_3A53_VH.tif'), vv=PosixPath('/home/jmherning/code/AI_Water/notebooks/data/working/S1A_IW_20191203T224543_DVP_RTC10_G_gpuned_3A53_VV.tif'))]


# Run: Create Mask

In [ ]:
# start = datetime(year=2019,month=12,day=3,hour=0,minute=0,second=0)
# end   = datetime(year=2019,month=12,day=4,hour=0,minute=0,second=0)

# api = hyp3_login()
# id = 2810

model = 'AI_MEKONG_CONSOLIDATED_FCN_512'
# mask_name = "Mekong_consolidated_test_notebook_dellater"
output_path = output_directory.relative_to(Path.cwd())
cnt = 1
for s in vvvh_paths:
    vv_path = Path(s.vv).relative_to(Path.cwd())
    vh_path = Path(s.vh).relative_to(Path.cwd())
    out = output_path / f"{vv_path.stem}_{cnt}.tif"
    create_water_mask(model, str(vv_path), str(vh_path), str(out))
    cnt += 1
    
    


# products = get_sub_products(api, id, start, end)
# product_urls = [p.url for p in products]
# pprint(f"urls to download={product_urls}")




In [140]:
Path('/home/jmherning/code/AI_Water/notebooks/data/working/S1A_IW_20191203T224518_DVP_RTC10_G_gpuned_54B9_VV.tif').relative_to('/home/jmherning/code/AI_Water/notebooks')

PosixPath('data/working/S1A_IW_20191203T224518_DVP_RTC10_G_gpuned_54B9_VV.tif')